In [8]:
import time
def timer(func):
  def wrapper(*args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    computation_time = end_time - start_time
    print(f"Executation time of {func.__name__}: {computation_time} seconds")
    return result
  return wrapper

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'


# MNIST 데이터셋을 이용한 이미지 분류
MNIST 데이터셋은 torchvision 라이브러리를 통해 불러올 수 있음

In [2]:
# Hyperparameter 설정
lr = 0.1
epochs = 100
min_batch = 64

In [3]:
import torch.utils.data as data
import torchvision

train_dataset = torchvision.datasets.MNIST(".", download=True, # torchvision의 datasets의 MNIST를 "." 현재 파일이 있는 경로에 다운로드 하겠다
                                           train=True, #test set(즉 val data) 가져오려면 False
                                           transform=torchvision.transforms.ToTensor()) #이미지를 그대로 가져오는게 아니라 tensor로 바꿔주기
test_dataset = torchvision.datasets.MNIST(".", download=True,
                                           train=False,
                                           transform=torchvision.transforms.ToTensor())

100%|██████████| 9912422/9912422 [00:00<00:00, 166404684.64it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 111748795.04it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 80885726.61it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 23007884.99it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



## 데이터로더 (DataLoader)
우리의 딥러닝 학습은 미니배치로 이루어지는데, 데이터셋을 training pipeline에 적절하게 쏴주는 녀석

In [4]:
from torch.utils.data.dataloader import DataLoader
# dataset 종류마다 DL 만들어야 함
train_loader = DataLoader(dataset=train_dataset, batch_size=min_batch, shuffle=True) # 매번 섞어줌. 순서를 학습할 수도 있어서...test는 굳이
test_loader = DataLoader(dataset=test_dataset, batch_size=min_batch, shuffle=False)


## 객체지향을 이용한 모델
모델 class를 정의해서 학습을 해보자!

Customized MLP 구현

In [5]:
class DeepClassification(nn.Module) :# n.Module을 상속 -> PyTorch에서 이미 만들어놓은 함수를 사용할 수 있게 됨
  def __init__ (self, hidden_variables=[128, 64, 10], input_output_dim=(28*28, 10)) : # 객체 안의 정보 정의. hidden_variables = 층의 개수. 이미지가 28*28로 들어옴. 10개는 (0~9)까지라서
    super().__init__()
    self.input_variable_dim = input_output_dim[0] #28*28
    self.output_variable_dim = input_output_dim[1] #10
    self.list_hidden_variable = hidden_variables
    self.layer = nn.Sequential() # Neural Net과 관련된 일련의 것을 집어넣을 거야! array = []와 같은 역할

    variable_dim = self.input_variable_dim # 안에서 동작하는 변수라서 self. 안붙임
    for i, hidden_variable in enumerate(self.list_hidden_variable) : # 0 : 128, 1: 64, 2 : 10
      self.layer.add_module('layer_' + str(i), nn.Linear(variable_dim, hidden_variable)) # 추가할 때 add_module (PyTorch)
                                                # 28*28, 128 -> 28*28, 64 -> ...
                                                # Wx+b. W : (28*28) * 128
                                                # img X는 원래 1 * (28*28)
                                                # Wx : 1* (28*28) * (28*28) * 128 = 1 * 128
      self.layer.add_module('custom_activation_'+str(i), nn.ReLU())
      variable_dim = hidden_variable # 새로 맞춰주는 역할.
    self.layer.add_module('dropout_' + str(i), nn.Dropout(0.1))
    self.layer.add_module('final_layer', nn.Linear(variable_dim, self.output_variable_dim)) # 64 * 10

  def forward(self, x) : # 입력. forward 이름 바꾸면 안돼!
    x = x.view(-1, self.input_variable_dim) # (1, (28*28)==784) 이미지 flatten
    y_hat = self.layer(x)
    return y_hat



In [6]:
def make_train_step(model, loss_fn, optimizer):
  def train_step_fn(x, y):
    model.train() # 모델을 학습모드로 전환
    y_hat = model(x)
    loss = loss_fn(y_hat, y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    return loss.item()
  return train_step_fn

In [9]:
@timer
def train_model_classification(epochs=1000, eval_test_accuracy=False):
  train_losses = []
  test_accuracies = []
  for epoch in range(epochs):
    mini_batch_losses = []
    for x_minibatch, y_minibatch in train_loader:
      x_minibatch = x_minibatch.to(device)
      y_minibatch = y_minibatch.to(device)
      mini_batch_loss = train_step(x_minibatch, y_minibatch)
      mini_batch_losses.append(mini_batch_loss)

    if (epoch + 1) % 5 == 0:
      loss = np.mean(mini_batch_losses) # cpu에 있기 때문에 numpy 사용
      print("train loss at {} epoch {}".format(epoch + 1, round(loss, 10)))
      train_losses.append(loss)

  model.eval()
  correct_predictions = 0
  total_samples = 0

  for x_minibatch, y_minibatch in test_loader:
    x_minibatch = x_minibatch.to(device)
    y_minibatch = y_minibatch.to(device)
    y_hat = model(x_minibatch)
    _, predicted_labels = torch.max(y_hat, 1)
    correct_predictions += (predicted_labels == y_minibatch).sum().item()
    total_samples += y_minibatch.size(0)

  accuracy = correct_predictions / total_samples
  print("테스트 정확도: ", accuracy)


  return train_losses


In [10]:
model = DeepClassification().to(device) # model을 GPU로 보내기
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # SGD 보다 Adam으로 하면 성능이 훨씬 좋아짐
train_step = make_train_step(model, loss_fn, optimizer)
train_loss = train_model_classification(epochs=2, eval_test_accuracy=True)

테스트 정확도:  0.9564
Executation time of train_model_classification: 16.92313838005066 seconds
